In [ ]:
import pyvacon
import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.models.plot as model_plot
import pyvacon.models.tools as model_tools
import pyvacon.analytics as analytics
import pyvacon.tools.converter as converter

from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Rectangle
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.transforms as mtransforms
%matplotlib inline

import datetime as dt
import math
import numpy as np

from scipy import stats
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import ipywidgets as widgets

# use ipynb to import function definitions from another notebook
import ipynb
from ipynb.fs.defs.ir_shock_scenarios import getShockedDiscountCurve

In [ ]:
# We define some constants which we'll use repeatedly throughout this notebook.
notebook_is_draft = True
color_main = 'tab:blue'
color_highlight = 'tab:orange'
color_graphblue = "#4e79a7"
color_histmarker = "#4e79a7"
color_histmarkerborder = "White"
grid_alpha = 0.4
default_daycounter_type = enums.DayCounter.ACTACT
default_interpolation_type = enums.InterpolationType.LINEAR
default_extrapolation_type = enums.ExtrapolationType.NONE
default_plotly_scatter_mode = 'lines'
if notebook_is_draft:
    default_sample_size_MC = 100
else:
    default_sample_size_MC = 10000
refdate = dt.datetime(year = 2019, month = 12, day = 30)

In [ ]:
# Same for functions
def get_default_title_dict(title_text):
    return dict(
        text = title_text,
        y = 0.9,
        x = 0.475,
        xanchor = 'center',
        yanchor = 'top')

# Introduction
## Value at risk
Value at risk (VaR) is a measure for the risk in a portfolio of financial assets. Given a time horizon of $n$ days and a confidence level $\alpha$, the VaR is the loss of value, which has the probability $\alpha$ not to be exceeded within the next $n$ days. In other words, the VaR is the $\alpha$-quantile of the distribution of loss in the value of a portfolio other the next $n$ days.

The different methods for estimating the value at risk can be put into two major categories: Those using analytical models and those using simulations.

The goal of **analytical** methods is to define a probability distribution, which approximates the actual probability distribution of the portfolio value. One can then write down a closed formula for the value at risk.

**Simulation**-based methods simulate the change in value over the next $n$ days and use the resulting relative frequency distribution to 'read off' the value at risk.

## Historical simulation
A very popular way of simulating changes in value uses past market data to estimate what will happen in the future. To do so, we first have to identify all market variables affecting the portfolio value. Then we collect data on how these variables moved over the past $k+n$ days. This allows us to calculate $k$ historical scenarios of what can happen in $n$ days. Assuming that the market will behave in the future as it did in the past, we can compute the portfolio value in each of these scenarios. This provides us with a relative frequency distribution, which we then use to determine the value at risk.

## Monte Carlo simulation
Monte Carlo simulation is similar to historical simulation in the sense that we also
- generate a set of market scenarios,
- compute the value of our portfolio in each of these scenarios and
- use the resulting relative frequency distribution to determine the value at risk.

They differ in the method for generating market scenarios: Instead of historical data, Monte Carlo simulation uses randomly generated movements of all relevant market variables. This requires more work (for example, you first have to develop a model for the market movements), but also comes with more flexibility.

## This notebook
We are going to look at two rather simple portfolios, one containing only a single bond and one containing a swap in addition to that bond. Their values can be computed by summing over all discounted future cash flows of said bond (and swap). Therefore, the only market variables affecting the portfolio values are the interest rates we use to determine the discount factors.
We will use both historical and Monte Carlo simulation to obtain interest rate scenarios. Based on these scenarios, we are going to determine the value at risk for both portfolios.

# Historical simulation
## Historical data
We choose to discount future cash flows using EONIA interest rate curves. We have historical data from every business day of 2018 and 2019 available to us. The data includes the actual over-night rates plus forward rates for various maturities. We'll load the data for maturities of 1 day, 1-11 months and 1-10 years.

*Note: These interest rates are not zero-coupon rates, but we are currently using them as is. Bootstrapping is still on the TODO list.*

In [ ]:
# load test data from an Excel file
xl = pd.ExcelFile('TestDaten.xlsx')
#print(xl.sheet_names)

In [ ]:
# import data into pandas dataframe
data_EONIA = xl.parse('EONIA')
data_EONIA = pd.DataFrame(data_EONIA,
                       columns = [
                           'Stichtag',
                           '1t',
                           '1m',
                           '2m',
                           '3m',
                           '4m',
                           '5m',
                           '6m',
                           '7m',
                           '8m',
                           '9m',
                           '10m',
                           '11m',
                           '1j',
                           '2j',
                           '3j',
                           '4j',
                           '5j',
                           '6j',
                           '7j',
                           '8j',
                           '9j',
                           '10j'
                       ])


# convert Excel dates to a more useful format and add them to the data frame as a new column
data_EONIA['datum'] = pd.TimedeltaIndex(data_EONIA['Stichtag'], unit='d') + dt.datetime(1899, 12, 30)
#display(data_EONIA.head(5))
#display(data_EONIA.tail(5))

Since we'll need them later, we store the selected maturities in the form of year fractions and dates (relative to our reference date defined above).

In [ ]:
# maturities in years
sampling_points_EONIA_yf = [1/365] # 1 day
sampling_points_EONIA_yf.extend( (np.arange(11)+1)/12 ) # 1 to 11 months
sampling_points_EONIA_yf.extend(np.arange(10)+1) # 1 to 10 years
#print(sampling_points_EONIA_yf)

# compute the maturity dates (using the reference date defined above)
sampling_points_EONIA_dates = []
sampling_points_EONIA_dates.append(refdate + dt.timedelta(days = 1))
for i in range(11):
    sampling_points_EONIA_dates.append(refdate + dt.timedelta(days = (i+1)*30))
for i in range(10):
    year = refdate.year + i + 1
    month = refdate.month
    day = refdate.day
    sampling_points_EONIA_dates.append(
        dt.datetime(year = year, month = month , day = day)
    )
#print(sampling_points_EONIA_dates)

## Scenario generation
As mentioned in the introduction, our goal is to use historical data to simulate how much the relevant market variables might change from now to $n$ days from now.

### Example 1
Let's assume that $n=1$. In that case, we are asking how much a given market variable can change from one business day to the next.
We assume that our historical data is ordered by date ascending and numbered consecutively, starting at 1. If $v_i$ denotes the value of the market variable on day $i$, then we can compute change scenarios in the following way.

| Scenario | From | To | Absolute change | Relative change |
| :---: | :---: | :---: | :---: | :---: |
| 1 | Day 1 | Day 2 | $d_1 = v_2 - v_1$ | $q_1 = \frac{v_2}{v_1}$ |
| 2 | Day 2 | Day 3 | $d_2 = v_3 - v_2$ | $q_2 = \frac{v_3}{v_2}$ |
| 3 | Day 3 | Day 4 | $d_3 = v_4 - v_3$ | $q_3 = \frac{v_4}{v_3}$ |
| 4 | Day 4 | Day 5 | $d_4 = v_5 - v_4$ | $q_4 = \frac{v_5}{v_4}$ |
| ... | ||||

After we compute these change scenarios (or shift scenarios), we can apply them to the current value $v$ of the market variable to obtain market scenarios: We can either add the absolute changes to the current value...

| Scenario | Value of market variable |
| :---: | :---: | 
| 1 | $v + d_1$ | 
| 2 | $v + d_2$ | 
| 3 | $v + d_3$ | 
| 4 | $v + d_4$ |
| ... | |

... or multiply the current value by the relative changes

| Scenario | Value of market variable |
| :---: | :---: | 
| 1 | $v \cdot q_1$ |
| 2 | $v \cdot q_2$ | 
| 3 | $v \cdot q_3$ | 
| 4 | $v \cdot q_4$ |
| ... | |

Which of these approaches you choose should depend on the considered market variable. In the case of interest rates, it turns out that using absolute changes produces more realistic scenarios than using relative changes.

### Example 2
Note that, since we have one data point for every business day, the way we computed the change scenarios in Example 1 seemed very natural. If we now let $n=10$, we have to think about it more carefully. Consider the following two approaches.

*Approach 1*

We compute the change in value from day $i$ to day $i+10$ for **all days** where that is possible.

| Scenario | From | To | Absolute Change | Relative Change |
| :---: | :---: | :---: | :---: | :---: |
| 1 | Day 1 | Day 11 | $v_{11} - v_1$ | $\frac{v_{11}}{v_1}$ |
| 2 | Day 2 | Day 12 | $v_{12} - v_2$ | $\frac{v_{12}}{v_2}$ |
| 3 | Day 3 | Day 13 | $v_{13} - v_3$ | $\frac{v_{13}}{v_3}$ |
| 4 | Day 4 | Day 14 | $v_{14} - v_4$ | $\frac{v_{14}}{v_4}$ |
| ... | ||||

You'll find that this leads to significant **overlap in the time frames** (From -> To) behind the scenarios. The time frames of scenario 2 and scenario 4, for example, overlap in days 4 to 12. This introduces **correlation** between the scenarios.

Remark: This is an example of **autocorrelation**. In the context of time series, autocorrelation is a measure of the similarity between values of one and the same variable at different points in time. In our example that variable is the change in interest rates in the last 10 days. By choosing overlapping time frames, each data point has an influence on multiple values in the time series. This leads to correlation between the values. The bigger the overlap, the stronger the correlation.


*Approach 2*

To avoid this effect, we can choose the time frames such that they have less or no overlap.

| Scenario | From | To | Absolute Change | Relative Change |
| :---: | :---: | :---: | :---: | :---: |
| 1 | Day 1 | Day 11 | $v_{11} - v_{1}$ | $\frac{v_{11}}{v_1}$ |
| 2 | Day 11 | Day 21 | $v_{21} - v_{11}$ | $\frac{v_{21}}{v_{11}}$ |
| 3 | Day 21 | Day 31 | $v_{31} - v_{21}$ | $\frac{v_{31}}{v_{21}}$ |
| 4 | Day 31 | Day 41 | $v_{41} - v_{31}$ | $\frac{v_{41}}{v_{31}}$ |
| ... | ||||

As a consequence, we end up with only about **a tenth the number of scenarios** we had in Approach 1. Of course, we can try to get more data, but that can be expensive or simply not possible (especially, if you consider time frames spanning a whole year, as is often the case in practice). Furthermore, one can argue that data becomes less relevant the further it reaches into the past. 


### What we do in this notebook
The following code is generic in the sense that you can freely choose the time horizon $n$ and whether you want the scenarios to be computed using absolute or relative changes. However, the amount of overlap in the time frames can currently not be controlled.

Assuming that our historical data is ordered by date ascending and numbered consecutively, let $n$ be the selected time horizon in days, $v$ be the current value of a market variable and $v_i$ the value it had on date $i$. Then we'll compute the value $s_i$ of the market variable in the $i$-th scenario as either
$$s_i = v + (v_{i+n} - v_i)$$
or
$$s_i = v \cdot \frac{v_{i+n}}{v_i}$$

In [ ]:
timehorizon = 1 # number of business days
scenario_construction_type = 'absolute' # absolute or relative

*Note: You can change these parameters to your liking and rerun the code to see the effects. You can use this to verify that the interest rate scenarios generated by applying relative changes can be a bit unrealistic.*

We now compute scenarios using both approaches. Afterwards, we choose which set of scenarios we're actually going to use (based on the constant defined above). We assume that the latest EONIA curve available to us is the same as the current curve.

In [ ]:
# restrict to the columns containing interest rates
data_EONIA_rates_only = pd.DataFrame(
                            data_EONIA,
                            columns = data_EONIA.columns[~data_EONIA.columns.isin(['Stichtag','datum'])]
                        )

# save the current market data in a pandas.series
data_EONIA_current = data_EONIA_rates_only.iloc[0,:]
#display(data_EONIA_current)

In [ ]:
# Compute absolute and relative changes

# Copy the data frame structure
data_scenarios_absolute = pd.DataFrame().reindex_like(data_EONIA_rates_only)
data_scenarios_relative = pd.DataFrame().reindex_like(data_EONIA_rates_only)

# Compute the values
n = timehorizon

for i in range(len(data_EONIA_rates_only.index) - n):
    for col in data_EONIA_rates_only.columns:
        data_scenarios_absolute.iloc[i + n, :][col] = data_EONIA_current[col] + data_EONIA_rates_only.iloc[i, :][col] - data_EONIA_rates_only.iloc[i + n, :][col]
        if data_EONIA_rates_only.iloc[i + n, :][col] != 0:
            data_scenarios_relative.iloc[i + n, :][col] = data_EONIA_current[col] * data_EONIA_rates_only.iloc[i, :][col] / data_EONIA_rates_only.iloc[i + n, :][col]

            
# Remove the rows containing NaN (i.e. the first n rows and those where we divided by 0)
data_scenarios_absolute = data_scenarios_absolute.dropna()
data_scenarios_relative = data_scenarios_relative.dropna()

In [ ]:
# Decide which scenarios to use in the rest of the notebook
if scenario_construction_type == 'relative':
    data_scenarios = data_scenarios_relative
if scenario_construction_type == 'absolute':
    data_scenarios = data_scenarios_absolute

# data_scenarios.describe()

## Plot Scenarios
To get a sense of how different the generated scenarios are from the current data, we plot all of them and highlight the ones that are (in a certain sense) the 'most distant'.

In [ ]:
# Compute the 'distances' of all scenarios to the current EONIA curve and sort them by that distance

considered_columns = [
    '1t',
#    '1m',
#    '2m',
   '3m',
#    '4m',
#    '5m',
   '6m',
#    '7m',
#    '8m',
#    '9m',
#    '10m',
#    '11m',
   '1j',
   '2j',
   '3j',
   '4j',
   '5j',
   '6j',
   '7j',
   '8j',
   '9j',
   '10j'
]
diffs = data_scenarios[considered_columns] - data_EONIA_current[considered_columns]
distances = [ np.linalg.norm(row, ord = 2) for index, row in diffs.iterrows() ]
data_scenarios_with_dist = data_scenarios.copy()
#print(data_scenarios_with_dist)
data_scenarios_with_dist['dist'] = distances
#print(distances)
data_scenarios_with_dist.sort_values(by = 'dist', ascending = False, inplace=True)
data_scenarios_with_dist = data_scenarios_with_dist.reset_index(drop=True)
data_scenarios_without_dist = data_scenarios_with_dist.drop('dist', axis=1)
#print(data_scenarios_with_dist)
#print(data_scenarios_with_dist.iloc[0:10])

# We'll highlight the 'most distant' scenarios in a different color in the plot below
indeces_most_distant = data_scenarios_with_dist.index.isin([0,1,2,3])

# We'll use this later
maxdist_hist = max(distances)

# clean up
# diffs.describe()
del diffs
del distances

In [ ]:
# Plot the scenarios

# use matplotlib
if False:
    fig = plt.figure(figsize=(16,8))
    ax = fig.gca()

    color_current = 'w'
    color_bulk = 'k'
    color_maxdist = 'tab:blue'
    ax.plot(sampling_points_EONIA_yf, data_EONIA_current, '.-', label = 'current EONIA curve', color = color_current, zorder = 20)
    ax.plot(sampling_points_EONIA_yf, data_scenarios_without_dist[~indeces_most_distant].transpose(), '.-', label = 'other scenarios', color = color_bulk, zorder = 15, alpha=0.05)
    ax.plot(sampling_points_EONIA_yf, data_scenarios_without_dist[indeces_most_distant].transpose(), '.-', label = 'extreme scenarios', color = color_maxdist, zorder = 15, alpha=1)

    plt.xlabel('Expiry (in years)')
    plt.ylabel('Interest rate (in percent)')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    legend_elements = [
        Patch(facecolor=color_current, edgecolor='gainsboro', label='current EONIA curve'),
        Patch(facecolor=color_maxdist, label='extreme scenarios'),
        Patch(facecolor=color_bulk, label='other scenarios')
    ]
    plt.legend(handles=legend_elements, loc='lower right')

    plt.show()

# use plotly
else:
    fig = go.Figure()

    # We'll highlight the 'most distant' scenarios in a different color in the plot below
    highlighted_indeces = [0,1,2,3]

    showlegend = True
    for i in data_scenarios_without_dist.index:
        if i not in highlighted_indeces:
            fig.add_trace(go.Scatter(
                x = sampling_points_EONIA_yf,
                y = data_scenarios_without_dist.iloc[i],
                name = 'other scenarios',
                legendgroup = 'other scenarios',
                mode = default_plotly_scatter_mode,
                showlegend = showlegend,
                line=dict(color="Black"),
                opacity = 0.08
            ))
            showlegend = False

    showlegend = True
    for i in highlighted_indeces:
        fig.add_trace(go.Scatter(
            x = sampling_points_EONIA_yf,
            y = data_scenarios_without_dist.iloc[i],
            name = 'extreme scenarios',
            legendgroup = 'extreme scenarios',
            mode = default_plotly_scatter_mode,
            showlegend = showlegend,
            line=dict(color=color_graphblue)
        ))
        showlegend = False

    fig.add_trace(go.Scatter(x = sampling_points_EONIA_yf, y = data_EONIA_current, name = 'current EONIA curve', mode = default_plotly_scatter_mode, line=dict(color="LightCyan"),))


    fig.update_layout(
        showlegend=True,
        xaxis = dict(title_text = "Expiry (in years)"),
        yaxis = dict(title_text = "Interest rate (in percent)"),
        legend = dict(traceorder='reversed'),
        title=get_default_title_dict("Historical scenarios")
    )

    fig.show()

In the case that the scenarios were constructed using relative historical changes, you'll probably find that some of them are rather extreme. To get a better understanding of why they are, we take a closer look at the scenarios containing the highest and the lowest interest rates found in any scenario.

In [ ]:
if scenario_construction_type == 'relative':
    # print(data_scenarios.min())
    # print(data_scenarios.idxmin())
    # print(data_scenarios.min().min())
    # print(data_scenarios.min().idxmin())
    # print(imin)

    imin = data_scenarios.idxmin()[data_scenarios.min().idxmin()]
    imax = data_scenarios.idxmax()[data_scenarios.max().idxmax()]

    display(
        pd.DataFrame({
            'Current': data_EONIA_current,
            'imin': data_EONIA_rates_only.loc[imin,:],
            'imin - n': data_EONIA_rates_only.loc[imin - timehorizon,:],
            'Scenario (imin)': data_scenarios.loc[imin,:],
            'imax': data_EONIA_rates_only.loc[imax,:],
            'imax - n': data_EONIA_rates_only.loc[imax - timehorizon,:],
            'Scenario (imax)': data_scenarios.loc[imax,:]
        }).head(len(data_EONIA_current))
    )

# Monte Carlo simulation


In [ ]:
# Define a function that handles plotting the scenarios
def PlotMonteCarloScenarios(monteCarloScenarios, useMatplotlib = True):
    
    if useMatplotlib:
        fig = plt.figure(figsize=(16,8))
        ax = fig.gca()
        color_current = 'w'
        color_bulk = 'k'
        ax.plot(sampling_points_EONIA_yf, data_EONIA_current, '.-', color = color_current, zorder = 20)
        ax.plot(sampling_points_EONIA_yf, monteCarloScenarios.transpose(), '.-', color = color_bulk, zorder = 15, alpha=20/len(monteCarloScenarios.index))

        plt.xlabel('Expiry (in years)')
        plt.ylabel('Interest rate (in percent)')

        legend_elements = [
            Patch(facecolor=color_current, edgecolor='gainsboro', label='current EONIA curve'),
            Patch(facecolor=color_bulk, label='Monte Carlo scenarios')
        ]
        plt.legend(handles=legend_elements, loc='lower right')

        plt.show()
    else:
        fig = go.Figure()

        showlegend = True
        opacity = max(0.001, min(1, 25/len(monteCarloScenarios.index)))
        for i in monteCarloScenarios.index:
            if i not in highlighted_indeces:
                fig.add_trace(go.Scatter(
                    x = sampling_points_EONIA_yf,
                    y = monteCarloScenarios.iloc[i],
                    name = 'other scenarios',
                    legendgroup = 'other scenarios',
                    mode = default_plotly_scatter_mode,
                    showlegend = showlegend,
                    line=dict(color="Black"),
                    opacity = opacity
                ))
                showlegend = False

        fig.add_trace(go.Scatter(x = sampling_points_EONIA_yf, y = data_EONIA_current, name = 'current EONIA curve', mode = default_plotly_scatter_mode, line=dict(color="LightCyan"),))


        fig.update_layout(
            showlegend=True,
            xaxis = dict(title_text = "Expiry (in years)"),
            yaxis = dict(title_text = "Interest rate (in percent)"),
            legend = dict(traceorder='reversed'),
            title={
                'text': "Monte Carlo scenarios",
                'y':0.9,
                'x':0.475,
                'xanchor': 'center',
                'yanchor': 'top'}
        )

        fig.show()

## Simple simulation
We apply a randomized parallel shift to the current interest rate curve.

In [ ]:
# Generate scenarios
n_sims_simpleMC = 2000#default_sample_size_MC
np.random.seed(7001)
random_shifts = np.random.normal(0, 0.02, n_sims_simpleMC)
data_scenarios_random_shift = pd.DataFrame().reindex_like(data_EONIA_rates_only)
data_scenarios_random_shift = data_scenarios_random_shift.iloc[0:0]

for x in random_shifts:
    data_scenarios_random_shift = data_scenarios_random_shift.append(data_EONIA_current + x, ignore_index=True)

# data_scenarios_random_shift.describe()

In [ ]:
PlotMonteCarloScenarios(data_scenarios_random_shift)
# PlotMonteCarloScenarios(data_scenarios_random_shift, False)

## Randomly picking historical data


### Using random distances

In [ ]:
# Generate scenarios
n_picks_dist = default_sample_size_MC
np.random.seed(7002)
random_dists = np.random.normal(0, maxdist_hist/3, n_picks_dist)
random_dists = [abs(d) for d in random_dists]

data_scenarios_random_pick_dist = pd.DataFrame().reindex_like(data_scenarios_with_dist)
data_scenarios_random_pick_dist = data_scenarios_random_pick_dist.iloc[0:0]

for d in random_dists:
    data_scenarios_random_pick_dist = data_scenarios_random_pick_dist.append(
        data_scenarios_with_dist.iloc[(data_scenarios_with_dist['dist']-d).abs().argsort()[:1]],
        ignore_index=True
    )
    
#print(maxdist_hist)
#data_scenarios_random_pick_dist.describe()

data_scenarios_random_pick_dist = data_scenarios_random_pick_dist.drop('dist', axis=1)

In [ ]:
# PlotMonteCarloScenarios(data_scenarios_random_pick_dist)

### Using random indeces

In [ ]:
# Generate scenarios
n_picks_indeces = default_sample_size_MC
np.random.seed(7003)
num_hist_scenarios = len(data_scenarios_without_dist.index)
random_indeces= []

while len(random_indeces) < n_picks_indeces:
    sample = math.floor(abs(np.random.normal(0, num_hist_scenarios/1)))
    if sample >= 0 and sample < num_hist_scenarios:
        random_indeces.append(sample)

data_scenarios_random_pick_indeces = pd.DataFrame().reindex_like(data_scenarios_without_dist)
data_scenarios_random_pick_indeces = data_scenarios_random_pick_indeces.iloc[0:0]

for i in random_indeces:
    data_scenarios_random_pick_indeces = data_scenarios_random_pick_indeces.append(
        data_scenarios_without_dist.iloc[num_hist_scenarios-i-1],
        ignore_index=True
    )

In [ ]:
# PlotMonteCarloScenarios(data_scenarios_random_pick_indeces)

### Using buckets
Instead of sampling from all historical interest rate curves available to us, we now divide them into buckets and choose one representative curve for each of them. Then, we take random samples from the set of buckets, instead of the entire set of historical data.

In [ ]:
# Create buckets
data_scenarios_buckets = pd.DataFrame().reindex_like(data_scenarios_without_dist)
data_scenarios_buckets = data_scenarios_buckets.iloc[0:0]

# The first bucket contains the current curve itself, i.e. the curve with distance 0
data_scenarios_buckets = data_scenarios_buckets.append(
    data_EONIA_current,
    ignore_index=True
)

n_hist_buckets_max = 60
# Then we increase the distance in steps of maxdist/n
k = len(data_scenarios_with_dist.index)-1 # data_scenarios_with_dist is ordered by dist descending
k_old = np.NaN
for i in range(1, n_hist_buckets_max-1):
    while data_scenarios_with_dist["dist"][k] < i * maxdist_hist/n_hist_buckets_max:
        k -= 1;
#     print(k+1)
    if k_old != k:
        data_scenarios_buckets = data_scenarios_buckets.append(
            data_scenarios_without_dist.iloc[k+1],
            ignore_index=True
        )
    k_old = k

# The last bucket is represented by the curve, which is the 'most distant' to the current curve
data_scenarios_buckets = data_scenarios_buckets.append(
    data_scenarios_without_dist.iloc[0],
    ignore_index=True
)
# print(len(data_scenarios_buckets.index))

In [ ]:
# Generate scenarios
n_picks_buckets = default_sample_size_MC
np.random.seed(7004)

data_scenarios_random_buckets = pd.DataFrame().reindex_like(data_scenarios_without_dist)
data_scenarios_random_buckets = data_scenarios_random_buckets.iloc[0:0]

while len(data_scenarios_random_buckets.index) < n_picks_buckets:
    sample = math.floor(abs(np.random.normal(0, len(data_scenarios_buckets.index)/2)))
    if sample >= 0 and sample < len(data_scenarios_buckets.index):
        data_scenarios_random_buckets = data_scenarios_random_buckets.append(
            data_scenarios_buckets.iloc[sample],
            ignore_index=True
        )

In [ ]:
# PlotMonteCarloScenarios(data_scenarios_random_buckets)

## Simulation via short rate models

In [ ]:
# todo
cir = analytics.CIRModel('CIR_Test', refdate, 1.0,0.05,0.20,0.025)
cir_dc = model_tools.compute_yieldcurve(cir, refdate,sampling_points_EONIA_dates)
hull_white = analytics.HullWhiteModel('HW_Test', refdate, 1, 0.1, cir_dc)


sim_dates = converter.createPTimeList(refdate, sampling_points_EONIA_dates)
refdate_LTime = converter.getLTime(refdate)
n_sims = 100
n_steps_per_year = 200
max_num_threads = 2

cir_lab = analytics.ModelLab(cir, refdate_LTime)
cir_lab.simulate(sim_dates, n_sims, n_steps_per_year, max_num_threads)

hw_lab = analytics.ModelLab(hull_white, refdate_LTime)
hw_lab.simulate(sim_dates, n_sims, n_steps_per_year, max_num_threads)

sampling_points_EONIA_datediffdays = [math.ceil(365*yf) for yf in sampling_points_EONIA_yf]
# dates = converter.createPTimeList(refdate, sampling_points_EONIA_datediffdays)

for i in range(n_sims):
    cir_lab.setFromSimulatedValues(cir, 1, i)  
    dc = model_tools.compute_yieldcurve(cir, sim_dates[0], sampling_points_EONIA_datediffdays)    
    mkt_plot.curve(dc, sim_dates, sim_dates[0], True, '', False)
    
    

In [ ]:
# todo
sampling_points_EONIA_datediffdays = [math.ceil(365*yf) for yf in sampling_points_EONIA_yf]
refdate_sim = sampling_points_EONIA_dates[0]
dates = converter.createPTimeList(refdate_sim, sampling_points_EONIA_datediffdays)

for i in range(n_sims):
    cir_lab.setFromSimulatedValues(cir, 1, i)  
    dc = model_tools.compute_yieldcurve(cir, refdate_sim, sampling_points_EONIA_datediffdays)    
    mkt_plot.curve(dc, dates, refdate_sim, True, '', False)
    

# for i in range(n_sims):
#     hw_lab.setFromSimulatedValues(hull_white, 1, i)  
#     dc = model_tools.compute_yieldcurve(hull_white, sim_times[0], dc_dates_p)    
#     mkt_plot.curve(dc, sim_dates, sim_times[0], True, '', False)
    

# Simple portfolio
To keep things simple, we start off with a portfolio containing only one fixed coupon bond with the following specifications:
 - It was issued on 2019/12/30
 - It has a maturity of 10 years
 - Its principal is 100€
 - It pays a 5€ coupon every year
 

In [ ]:
#define_bond
maturity = 10
principal = 100.0
coupon_rate = 0.05
maturity_date = dt.datetime(year = refdate.year + maturity, month = refdate.month, day = refdate.day)
#print(refdate)
#print(maturity_date)

# Generate the coupon payment schedule as a vector of datetimes
coupon_dates = []
for i in range(maturity):
    coupon_dates.append(dt.datetime(year = refdate.year + i + 1, month = refdate.month, day = refdate.day))
#print(coupon_dates)
coupon_rates = [coupon_rate]*len(coupon_dates)
coupon_payments = [coupon_rate*principal]*len(coupon_dates)

# We now use these specifications to define a fixed coupon bond
fixed_coupon_bond = pyvacon.instruments.BondSpecification('Fixed_Coupon', 'DBK', enums.SecuritizationLevel.NONE, 'EUR',
    maturity_date, refdate, principal, default_daycounter_type, coupon_dates, coupon_rates, '', [], [])

The current value of this portfolio can be computed by simply summing over all discounted future cash flows. Therefore, the only market variables affecting this value are the interest rates we use to determine the discount factors.

## Compute the Credit Spread and Portfolio Values

In [ ]:
# Define the pricer, we're going to use to price our bond
pricing_data_simple = pyvacon.pricing.BondPricingData()
pricing_data_simple.param = pyvacon.pricing.BondPricingParameter()
pricing_data_simple.param.useJLT = False
pricing_data_simple.pricingRequest = pyvacon.pricing.PricingRequest()
pricing_data_simple.pricingRequest.setCleanPrice(True)
pricing_data_simple.pricer = 'BondPricer'
pricing_data_simple.spec = fixed_coupon_bond

valdate = refdate # + dt.timedelta(days = timehorizon)
pricing_data_simple.valDate = valdate

Note: We are currently not taking portfolio aging into account: In the computations below, we are using the reference date as valuation date. That is, we look at the effects our shift scenarios would have on the value of our portfolio, if they were to happen instantaneously (instead of over the next $n$ days).

### Compute Credit Spread
**Credit spread** is the difference in yield between two investments of similar maturities, but different credit qualities. It can be interpreted as the risk premium for one investment over the other.

Given the low EONIA rates, the bond we defined above currently has a much higher yield than a hypothetical bond paying EONIA rates on the same principal. Pricing the bond using discount factors based on EONIA rates would grossly overestimate its value. That is, the price we compute would be a lot higher than the bond's actual market value. To avoid this, we determine the constant shift we need to apply to the interest rates used for discounting in order for our price to equal the market value of the bond.

In [ ]:
# Use the current EONIA rates + a constant rate to compute the price of the fixed coupon bond
# Vary the constant rate and repeat until the value of the bond is right about the same as its principal
creditspread = coupon_rate * 100 # in basis points
stepsize = coupon_rate * 100 # the initial step size used to vary the interest rate
spreads = []
values = []
for k in range(20):
    # create DC defined by the scenario
    dsc_fac = analytics.vectorDouble()
    spreadScenario = data_EONIA_current + creditspread;
    for i in range(len(spreadScenario)):
        dsc_fac.append(math.exp(-spreadScenario.iloc[i]/100*i)) # t = i years  # market data is given in basis points -> /100  
            
    discountCurve = analytics.DiscountCurve('dc_linear', refdate, sampling_points_EONIA_dates, dsc_fac, enums.DayCounter.ACTACT, enums.InterpolationType.LINEAR, enums.ExtrapolationType.NONE)
    pricing_data_simple.discountCurve = discountCurve
    
    results = pyvacon.pricing.price(pricing_data_simple)
    
    values.append(results.getPrice())
    spreads.append(creditspread)
    
    if values[k] > principal:
        creditspread += stepsize
    else:
        creditspread -= stepsize
    stepsize /= 2

#print(spreads)
#print(values)
#print(creditspread)
    

The credit spread is {{round(creditspread, 3)}}%.

In [ ]:
# Compute the current value without taking the credit spread into account

# Create DC defined by the scenario
dsc_fac = analytics.vectorDouble()
for i in range(len(data_EONIA_current)):
        dsc_fac.append(math.exp(-data_EONIA_current.iloc[i]/100*i)) # t = i years  # market data is given in basis points -> /100  

discountCurve = analytics.DiscountCurve('dc_linear', refdate, sampling_points_EONIA_dates, dsc_fac, default_daycounter_type, default_interpolation_type, default_extrapolation_type)
pricing_data_simple.discountCurve = discountCurve
results = pyvacon.pricing.price(pricing_data_simple)
current_value_without_credit_spread = results.getPrice()

In [ ]:
# Compute the current value

# Create DC defined by the scenario
dsc_fac = analytics.vectorDouble()
for i in range(len(data_EONIA_current)):
        dsc_fac.append(math.exp(-(data_EONIA_current + creditspread).iloc[i]/100*i)) # t = i years  # market data is given in basis points -> /100  

discountCurve = analytics.DiscountCurve('dc_linear', refdate, sampling_points_EONIA_dates, dsc_fac, default_daycounter_type, default_interpolation_type, default_extrapolation_type)
pricing_data_simple.discountCurve = discountCurve
results = pyvacon.pricing.price(pricing_data_simple)
currentPriceDirty = results.getPrice()
currentPriceClean = results.getCleanPrice()
#print(currentPriceDirty)
#print(currentPriceClean)

By taking into account the credit spread we computed above, we arrive at a current value of {{round(currentPriceDirty, 6)}}, which closely matches the actual market value of 100. If we didn't take the credit spread into account, we would arrive at a value of {{round(current_value_without_credit_spread, 2)}}.

### Compute Portfolio Values

In [ ]:
# Choose the scenarios that we're going to use
data_scenarios = data_scenarios_absolute
#data_scenarios = data_scenarios_random_buckets
compare_scenarios = True
#data_scenarios_compare = data_scenarios_random_pick_indeces
#data_scenarios_compare = data_scenarios_random_pick_dist
data_scenarios_compare = data_scenarios_random_shift
#data_scenarios_compare = data_scenarios_random_buckets
#data_scenarios_compare = data_scenarios_buckets

In [ ]:
# Compute the price of the fixed coupon bond at the valuation date defined above
# Repeat for every scenario
def ComputeValuesOfSimplePortfolio(refdate, sampling_points_dates, data_scenarios, creditspread, daycounter_type, interpolation_type, extrapolation_type):
    results_dirty = []
    results_clean = []
    for index, scenario in data_scenarios.iterrows():
        # add the credit spread we computed for our bond
        scenario = scenario + creditspread

        # create DC defined by the scenario
        dsc_fac = analytics.vectorDouble()
        for i in range(len(scenario)):
                dsc_fac.append(math.exp(-scenario.iloc[i]/100*i)) # t = i years  # market data is given in basis points -> /100  

        discountCurve = analytics.DiscountCurve('dc_linear', refdate, sampling_points_dates, dsc_fac, daycounter_type, interpolation_type, extrapolation_type)
        pricing_data_simple.discountCurve = discountCurve

        results = pyvacon.pricing.price(pricing_data_simple)
        results_dirty.append(results.getPrice())
        results_clean.append(results.getCleanPrice())
        #print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))
    return [results_dirty, results_clean]
    

results_dirty, results_clean = ComputeValuesOfSimplePortfolio(refdate, sampling_points_EONIA_dates, data_scenarios, creditspread, default_daycounter_type, default_interpolation_type, default_extrapolation_type)
    
if compare_scenarios:
    results_dirty_compare, results_clean_compare = ComputeValuesOfSimplePortfolio(refdate, sampling_points_EONIA_dates, data_scenarios_compare, creditspread, default_daycounter_type, default_interpolation_type, default_extrapolation_type)
    

In [ ]:
# In case we want to compare scenarios, we determine the minimal and maximal x-values to display in upcoming plots
if compare_scenarios:
    xmin = min(min(results_dirty), min(results_dirty_compare))
    xmax = max(max(results_dirty), max(results_dirty_compare))
    
    #minIndex = results_dirty.index(min(results_dirty))
    #print(minIndex)
    #print(results_dirty[minIndex])
    #print(data_scenarios.iloc[minIndex])

    #maxIndex = results_dirty.index(max(results_dirty))
    #print(maxIndex)
    #print(results_dirty[maxIndex])
    #print(data_scenarios.iloc[maxIndex])

    #results_series = pd.Series(results_dirty)
    #display(results_series.describe())

## Plot pricing results

In [ ]:
# Define a function that plots the histrograms
def plotHistogram(
    data,
    binsstart,
    binsend,
    nbins,
    title_xaxis,
    markercolor = color_histmarker,
    bordercolor = color_histmarkerborder
):
    xbins = dict(start = binsstart, end = binsend, size = (binsend-binsstart)/nbins)
    marker=dict(
        color=markercolor,
        line = dict(color = bordercolor, width = 1)
    )

    fig = go.Figure()
    fig.add_trace(go.Histogram(x = data, xbins = xbins, marker = marker))

    fig.update_layout(
        showlegend=False,
        xaxis = dict(title_text = title_xaxis, range = [binsstart, binsend]),
        yaxis = dict(title_text = "Number of occurences")
    #     ,title={
    #         'text': "Historical scenarios",
    #         'y':0.9,
    #         'x':0.475,
    #         'xanchor': 'center',
    #         'yanchor': 'top'}
    )

    fig.show()  

In [ ]:
# Histogramm of the pricing results
plotHistogram(
    data = results_dirty,
    binsstart = xmin,
    binsend = xmax,
    nbins = 60,
    title_xaxis = "Portfolio value"
)

if compare_scenarios:
    plotHistogram(
        data = results_dirty_compare,
        binsstart = xmin,
        binsend = xmax,
        nbins = 60,
        title_xaxis = "Portfolio value"
    )

In [ ]:
# Histogramm of the changes/differences in value
valDiffsDirty = np.asarray([res - currentPriceDirty for res in results_dirty])
binsstart_simple = xmin - currentPriceDirty
binsend_simple = xmax - currentPriceDirty
plotHistogram(
    data = valDiffsDirty,
    binsstart = binsstart_simple,
    binsend = binsend_simple,
    nbins = 60,
    title_xaxis = "Change in portfolio value"
)


if compare_scenarios:
    valDiffsDirty_compare = np.asarray([res - currentPriceDirty for res in results_dirty_compare])
    plotHistogram(
        data = valDiffsDirty_compare,
        binsstart = binsstart_simple,
        binsend = binsend_simple,
        nbins = 60,
        title_xaxis = "Change in portfolio value"
    )

## Compute Value at Risk

In [ ]:
# determine quantile
valDiffsDirty = (-1)*np.sort((-1)*valDiffsDirty)
quantile = 0.99
#print(np.quantile(valDiffsDirty, 1-quantile, interpolation='higher')) # apparently always uses ascending order

# Compute the number of the entry corresponding to the quantile defined above
quantileIndex = np.ceil(len(valDiffsDirty)*quantile).astype(int)
#print(quantileIndex)
#print(quantile * len(valDiffsDirty))

# To get the index of this entry, we have to subtract 1
quantileIndex -= 1

# Check correctness
#print('--------------')
#print(valDiffsDirty[quantileIndex-1])
#print((quantileIndex)/len(valDiffsDirty))
#print('--------------')
#print(valDiffsDirty[quantileIndex])
#print((quantileIndex + 1)/len(valDiffsDirty))
#print('--------------')
#print(valDiffsDirty[quantileIndex+1])
#print((quantileIndex + 2)/len(valDiffsDirty))
#print('--------------')


In [ ]:
# Plot cummulative relative frequencies of loss of portfolio value
losses = -valDiffsDirty

marker=dict(
    color=color_histmarker
)

fig = go.Figure()
fig.add_trace(go.Histogram(x = losses, nbinsx = len(losses)*2, marker = marker, cumulative_enabled = True, histnorm = 'probability'))

fig.update_layout(
    showlegend=False,
    xaxis = dict(title_text = "Loss of portfolio value"),
    yaxis = dict(title_text = "Cumulative relative frequency")
#     ,title={
#         'text': "Historical scenarios",
#         'y':0.9,
#         'x':0.475,
#         'xanchor': 'center',
#         'yanchor': 'top'}
)

fig.show()  

With a probability of {{round((quantileIndex + 1)/len(valDiffsDirty)*100, 3)}}% the value of our portfolio is not going to shrink by more than {{round(-1 * valDiffsDirty[quantileIndex], 4)}} in the next {{timehorizon}} day(s).

# Extended portfolio
## Add a swap
We swap the fixed coupon payments for interest payments based on EONIA rates.

In [ ]:
# Define the swap scpecification
startdates = [refdate]
startdates.extend(coupon_dates[0:len(coupon_dates)-1])
#startdates = converter.createPTimeList(refdate, startdates)

enddates = coupon_dates
#enddates = converter.createPTimeList(enddates, startdates)

#print(startdates)
#print(enddates)

paydates = enddates
resetdates = startdates

notionals = analytics.vectorDouble()
notionals.append(principal)

fixedleg = analytics.IrFixedLegSpecification(coupon_rate, notionals, startdates, enddates, paydates,'EUR', default_daycounter_type)

floatleg = analytics.IrFloatLegSpecification(notionals, resetdates, startdates, enddates,
                                    paydates,'EUR', 'test_udl', default_daycounter_type, 
                                    0)
                                    #creditspread/100) # spread is given in basis points

ir_swap = analytics.InterestRateSwapSpecification('TEST_SWAP', 'DBK', enums.SecuritizationLevel.COLLATERALIZED, 'EUR',
                                           converter.getLTime(paydates[-1]), fixedleg, floatleg)


### Recompute the value of our portfolio in all scenarios

In [ ]:
# Specify all data we need to price the swap
ir_swap_pricing_data = analytics.InterestRateSwapPricingData()

pay_leg_pricing_data = analytics.InterestRateSwapLegPricingData()
pay_leg_pricing_data.spec = ir_swap.getPayLeg()
pay_leg_pricing_data.fxRate = 1.0
pay_leg_pricing_data.weight = -1.0

rec_leg_pricing_data = analytics.InterestRateSwapFloatLegPricingData()
rec_leg_pricing_data.spec = ir_swap.getReceiveLeg()
rec_leg_pricing_data.fxRate = 1.0
rec_leg_pricing_data.weight = 1.0

ir_swap_pricing_data.pricer = 'InterestRateSwapPricer'
ir_swap_pricing_data.pricingRequest = analytics.PricingRequest()
ir_swap_pricing_data.valDate = converter.getLTime(refdate)
ir_swap_pricing_data.setCurr('EUR')
ir_swap_pricing_data.addLegData(pay_leg_pricing_data)
ir_swap_pricing_data.addLegData(rec_leg_pricing_data)

In [ ]:
# Compute the price of our portfolio
# Repeat for every scenario
results_dirty = []
results_clean = []
for index, scenario in data_scenarios.iterrows():
    # add the credit spread we computed for our bond
    
    # create DC defined by the scenario
    factorsEONIA = analytics.vectorDouble()
    factorsWithSpread = analytics.vectorDouble()
    for i in range(len(scenario)):
        factorsEONIA.append(math.exp(-scenario.iloc[i]/100*i)) # t = i years  # market data is given in basis points -> /100  
        factorsWithSpread.append(math.exp(-(scenario.iloc[i] + creditspread)/100*i)) # t = i years  # market data is given in basis points -> /100  
            
    dcEONIA = analytics.DiscountCurve('dc_linear', refdate, sampling_points_EONIA_dates, factorsEONIA, default_daycounter_type, default_interpolation_type, default_extrapolation_type)
    dcWithSpread   = analytics.DiscountCurve('dc_linear_spread', refdate, sampling_points_EONIA_dates, factorsWithSpread, default_daycounter_type, default_interpolation_type, default_extrapolation_type)
    
    pricing_data_simple.discountCurve = dcEONIA # dcWithSpread
    pay_leg_pricing_data.discountCurve = dcEONIA
    rec_leg_pricing_data.discountCurve = dcEONIA
    rec_leg_pricing_data.fixingCurve = dcEONIA
    
    prBond = pyvacon.pricing.price(pricing_data_simple)
    prSwap = analytics.price(ir_swap_pricing_data)
    dirty = prBond.getPrice() + prSwap.getPrice()
    clean = prBond.getCleanPrice() + prSwap.getCleanPrice()
    results_dirty.append(dirty)
    results_clean.append(clean)
    #print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))
#print(results_dirty)

### Compute the current value as a reference

In [ ]:
# Define a discount curve based on the current EONIA rates
factorsEONIA = analytics.vectorDouble()
factorsWithSpread = analytics.vectorDouble()
for i in range(len(data_EONIA_current)):
    factorsEONIA.append(math.exp(-data_EONIA_current.iloc[i]/100*i)) # t = i years  # market data is given in basis points -> /100  
    factorsWithSpread.append(math.exp(-(data_EONIA_current.iloc[i] + creditspread)/100*i)) # t = i years  # market data is given in basis points -> /100  
    
dcEONIA = analytics.DiscountCurve('dc_linear', refdate, sampling_points_EONIA_dates, factorsEONIA, default_daycounter_type, default_interpolation_type, default_extrapolation_type)
dcWithSpread = analytics.DiscountCurve('dc_linear_spread', refdate, sampling_points_EONIA_dates, factorsWithSpread, default_daycounter_type, default_interpolation_type, default_extrapolation_type)

pricing_data_simple.discountCurve = dcEONIA # dcWithSpread
pay_leg_pricing_data.discountCurve = dcEONIA
rec_leg_pricing_data.discountCurve = dcEONIA 
rec_leg_pricing_data.fixingCurve = dcEONIA

# compute portfolio value
prBond = pyvacon.pricing.price(pricing_data_simple)
prSwap = analytics.price(ir_swap_pricing_data)
#print(prSwap.getPrice())
#print(prBond.getPrice())
currentValueBond = prBond.getPrice()
currentValueSwap = prSwap.getPrice()
currentValue = prBond.getPrice() + prSwap.getPrice()
#print(currentValueSwap)
#print(currentValueBond)
#print(currentValue)

### Plot the pricing results

In [ ]:
# Histogramm of the changes/differences in value
valDiffsDirty = np.asarray([res - currentValue for res in results_dirty])
plotHistogram(
    data = valDiffsDirty,
    binsstart = binsstart_simple,
    binsend = binsend_simple,
    nbins = 60,
    title_xaxis = 'Change in portfolio value'
)

We can see that the swap we added to our portfolio cancels out any market risk, setting the value at risk to 0.

# Interest Rate Shock Scenarios

In [ ]:
# define the parameters for the shock scenarios

shockParams = pd.DataFrame({'Currency': [], 'Parallel': [], 'Short': [], 'Long': []})
shockParams = shockParams.append({'Currency': 'EUR', 'Parallel': 200, 'Short': 250, 'Long': 100}, ignore_index = True)
shockParams = shockParams.append({'Currency': 'GBP', 'Parallel': 250, 'Short': 300, 'Long': 150}, ignore_index = True)
shockParams = shockParams.append({'Currency': 'USD', 'Parallel': 200, 'Short': 300, 'Long': 150}, ignore_index = True)

## Compute the change in value

In [ ]:
# Compute the price of our portfolio
# Repeat for every scenario
results_dirty = []
results_clean = []
results_dirty_bondonly = []
results_clean_bondonly = []

currency = 'EUR'
parallel = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Parallel']
short = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Short']
long = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Long']
    
shockScenarios = ['ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen']

for shockScenario in shockScenarios:
    dcEONIA = getShockedDiscountCurve(
        'dc_linear',
        refdate,
        sampling_points_EONIA_dates,
        data_EONIA_current*100,
        default_daycounter_type,
        default_interpolation_type,
        default_extrapolation_type,
        shockScenario,
        parallel,
        short,
        long
    )
    
#     dcWithSpread = getShockedDiscountCurve(
#          'dc_linear_spread',
#          refdate,
#          sampling_points_EONIA_dates,
#          data_EONIA_current + creditspread,
#          default_daycounter_type,
#          default_interpolation_type,
#          default_extrapolation_type,
#          shockScenario,
#          parallel/100,
#          short/100,
#          long/100
#      )
    
    pricing_data_simple.discountCurve = dcEONIA # dcWithSpread
    pay_leg_pricing_data.discountCurve = dcEONIA
    rec_leg_pricing_data.discountCurve = dcEONIA
    rec_leg_pricing_data.fixingCurve = dcEONIA
    
    prBond = pyvacon.pricing.price(pricing_data_simple)
    prSwap = analytics.price(ir_swap_pricing_data)
    dirty = prBond.getPrice() + prSwap.getPrice()
    clean = prBond.getCleanPrice() + prSwap.getCleanPrice()
    results_dirty.append(dirty)
    results_clean.append(clean)
    results_dirty_bondonly.append(prBond.getPrice())
    results_clean_bondonly.append(prBond.getCleanPrice())
    #print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))
#print(results_dirty)

## Plot the change in value (comparison between our entire portfolio and the bond on its own)

In [ ]:
# Bar plot of the changes/differences in value
valDiffsDirty = np.asarray([res - currentValue for res in results_dirty])
valDiffsDirtyBondOnly = np.asarray([res - currentValueBond for res in results_dirty_bondonly])

y_shockscenarios_simple = valDiffsDirtyBondOnly/currentValueBond*100
ymin = min(y_shockscenarios_simple)
ymax = max(y_shockscenarios_simple)
ydiff = abs(ymax-ymin)
rangemin = ymin - ydiff/10
rangemax = ymax + ydiff/10

marker=dict(
    color=color_histmarker,
    line = dict(color = color_histmarker, width = 1)
)



# Plot results for the simple portfolio

fig = go.Figure()
fig.add_trace(go.Bar(x=shockScenarios, y=y_shockscenarios_simple, marker = marker))

fig.update_layout(
    showlegend=False,
    xaxis = dict(title_text = "Shock Scenario"),
    yaxis = dict(title_text = "Change in portfolio value [%]", range = [rangemin, rangemax])
    ,title=get_default_title_dict("Simple Portfolio")
)

fig.show()  



# Plot results for the extended portfolio

fig = go.Figure()
fig.add_trace(go.Bar(x=shockScenarios, y=valDiffsDirty/currentValue*100, marker = marker))

fig.update_layout(
    showlegend=False,
    xaxis = dict(title_text = "Shock Scenario"),
    yaxis = dict(title_text = "Change in portfolio value [%]", range = [rangemin, rangemax])
    ,title=get_default_title_dict("Extended Portfolio")
)

fig.show()  


del y_shockscenarios_simple, ymin, ymax, ydiff, rangemin, rangemax

# TODO

- Bootstrap zero-coupon interest rates
- 'Einheit' der Zinssaetze mit an die Funktionen uebergeben (?) (dezimal, percent, basispoint)
- Das Wort 'current' als Beschreibung für die jüngste in den Marktdaten vorhandene EONIA-Kurve ueberdenken
- Actually order the historical data by date ascending (instead of descending) to avoid confusion
- Illustrate the definition of VaR
- Portfolio mit zwei Bonds unterschiedlicher Laufzeiten untersuchen